# Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
# disable tensorflow log level infos
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # show only errors

import sys
import pandas as pd

if '../../../../notebooks/' not in sys.path:
    sys.path.append('../../../../notebooks/')
if 'src' not in sys.path:
    sys.path.insert(0, 'src')

import utils.constants as cts

from data_loaders.data_loader import DLName
from gt_loaders.gt_names import GTName
from exp_runner import ExperimentRunner
from model_evaluator import DataSource, DataPredSelection
from base_models import BaseModel
from optimizers import Optimizer
from model_creator import MTLApproach, NAS_MTLApproach

 ==> Restrict GPU memory growth: True


# Start Network runner

In [3]:
kwargs = { 
    'use_neptune': False,
    'exp_params' : {
        'name': 'train_vgg16',
        'description': 'Training mtl network for ALL requisites - 10 epochs - without early_stopping in FVC GT dataset NOT aligned',
        'tags': ['vgg16', 'ground truths', 'adamax', 'mtl', '10 epochs'],
        'src_files': ["src/**/*.py"]
    },
    'properties': {
        'approach': NAS_MTLApproach.APPROACH_1,
        'reqs': list(cts.ICAO_REQ),
        'aligned': True,
        'use_gt_data': True,
        'gt_names': {
            'train_validation': [],
            'test': [],
            'train_validation_test': [GTName.FVC]
        },
        'balance_input_data': False,
        'train_model': True,
        'save_trained_model': False,
        'orig_model_experiment_id': '',
        'sample_training_data': False,
        'sample_prop': 1.0
    },
    'net_train_params': {
        'base_model': BaseModel.VGG16,
        'batch_size': 32,
        'n_epochs': 10,
        'early_stopping': 10,
        'learning_rate': 1e-3,
        'optimizer': Optimizer.ADAMAX,
        'dropout': 0.3
    },
    'nas_params': {
        'max_blocks_per_branch': 3,
        'max_train_steps_per_op': 5
    }
}

runner = ExperimentRunner(**kwargs)

-------------------- Init ExperimentRunner -------------------
---------------------------
Parent Process ID: 95416
Process ID: 110891
---------------------------
-----
Use Neptune:  False
-----
-------------------
Args: 
{'exp_params': {'description': 'Training mtl network for ALL requisites - 10 '
                               'epochs - without early_stopping in FVC GT '
                               'dataset NOT aligned',
                'name': 'train_vgg16',
                'src_files': ['src/**/*.py'],
                'tags': ['vgg16',
                         'ground truths',
                         'adamax',
                         'mtl',
                         '10 epochs']},
 'nas_params': {'max_blocks_per_branch': 3, 'max_train_steps_per_op': 5},
 'net_train_params': {'base_model': <BaseModel.VGG16: {'target_size': (224, 224), 'prep_function': <function preprocess_input at 0x7f9de4cd2ee0>}>,
                      'batch_size': 32,
                      'dropout': 0.3,
 

# Load Data

In [4]:
runner.load_training_data()

-------------------- load training data -------------------
Loading data
Loading GT FVC - TRAIN split...
..Ignoring 0 empty label values
Input data.shape: (4926, 26)
Loading GT FVC - VALIDATION split...
..Ignoring 0 empty label values
Input data.shape: (547, 26)
Loading GT FVC - TEST split...
..Ignoring 0 empty label values
Input data.shape: (288, 26)
Data loaded


# Producing Fake Data

In [7]:
runner.produce_fake_data()

-------------------- producing fake data for experimental purposes -------------------
fake_train_data.shape: (500, 26)
fake_validation_data_df.shape: (100, 26)
fake_test_data_df.shape: (50, 26)


# Data Generators

In [8]:
runner.setup_data_generators()

-------------------- setup data generators -------------------
Starting data generators
Found 500 validated image filenames.
Found 100 validated image filenames.
Found 50 validated image filenames.
TOTAL: 650

Logging class indices
 .. MTL model not logging class indices!

Logging class labels
 COMPLIANT label: 1
 NON_COMPLIANT label: 0
 DUMMY label: -1
 DUMMY_CLS label: 2
 NO_ANSWER label: -99


# Neural Architecture Search

In [11]:
runner.run_neural_architeture_search()

-------------------- run neural architecture search -------------------
++++++++++++++++++++++++++++++++++++++++++++++++++ NEW TRAIN ++++++++++++++++++++++++++++++++++++++++++++++++++
 ----- Training 0 | Config: {'n_denses_0': 1, 'n_denses_1': 2, 'n_denses_2': 5, 'n_denses_3': 5} --------
Creating model...
Model created
Training VGG16 network
 .. Not fine tuning base model...
  .. Total params: 15,714,094
  .. Trainable params: 999,406
  .. Non-trainable params: 14,714,688
Epoch 1/2
15/15 [==============================] - 11s 383ms/step - loss: 3.6311 - background_loss: 3.2167 - close_loss: 2.1989 - ink_mark_loss: 1.4295 - pixelation_loss: 2.9830 - washed_out_loss: 2.0499 - blurred_loss: 3.9921 - sh_head_loss: 2.4689 - mouth_loss: 1.7861 - veil_loss: 2.3359 - red_eyes_loss: 1.1364 - flash_lenses_loss: 0.8229 - dark_glasses_loss: 1.1734 - l_away_loss: 0.7677 - frame_eyes_loss: 0.8225 - hair_eyes_loss: 0.9104 - eyes_closed_loss: 0.9974 - frames_heavy_loss: 1.0568 - sh_face_loss: 0.8157 

{'model_0': {'final_EER_mean': 0.4968727272727273,
  'final_ACC': 0.5031818181818183},
 'model_1': {'final_EER_mean': 0.5, 'final_ACC': 0.4918181818181818},
 'model_2': {'final_EER_mean': 0.5022818181818182,
  'final_ACC': 0.4904545454545454},
 'model_3': {'final_EER_mean': 0.5, 'final_ACC': 0.4918181818181818},
 'model_4': {'final_EER_mean': 0.4975136363636363,
  'final_ACC': 0.5022727272727273}}

# Testing Network Modification